<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM many-to-many

### Datos
El objecto es utilizar una serie de sucuencias númericas (datos sintéticos) para poner a prueba el uso de las redes LSTM. Este ejemplo se inspiró en otro artículo, lo tienen como referencia en el siguiente link:\
[LINK](https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras-part-2/)

In [ ]:
import re

import numpy as np
import pandas as pd

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

In [ ]:
# Generar datos sintéticos
X = list()
y = list()

# En ambos casos "X" e "y" son vectores de números de 5 en 5
X = [x for x in range(5, 301, 5)]
y = [x+15 for x in X]

print(f"datos X (len={len(X)}):", X)
print(f"datos y (len={len(y)}):", y)

In [ ]:
# Se desea agrupar los datos de a 3 elementos
X = np.array(X).reshape(len(X)//3, 3, 1)
y = np.array(y).reshape(len(y)//3, 3, 1)
print("datos X[0:2]:", X[0:2])
print("datos y[0:2]:", y[0:2])

In [ ]:
# Verificamos que la secuencia de entrada es igual a la secuencia de salida
# en cuanto a dimensiones
# Tendremos:
#  --> veinte grupos de datos (rows) (20)
#  --> cada grupo compuesto por tres elementos (3)
#  --> cada elemento representado en una sola dimension (1)
print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# Cardinalidad (cantidad de elementos distintos en el dataset)
data = np.append(X, y)
len(np.unique(data))

### 2 - Entrenar el modelo

In [ ]:
input_shape = X[0].shape
input_shape

In [ ]:
output_shape = y[0].shape
output_shape

In [ ]:
model = Sequential()

# input LSTM layer
# Aquí se transformar las entradas en features
# Retornarmos la secuencia para que la salida tenga la siguiente dimension:
#   --> (tamaño batch, tamaño serie, tamaño elemento)
# A diferencia de otra veces, estamos agregando el tamaño de la secuencia/serie
model.add(LSTM(128, activation='relu', input_shape=(input_shape), return_sequences=True))

# Al final tengo una salida (una secuencia) de 3 elementos juntos, 
# cada elemento de dimension 1:
# --> (3x1)
model.add(Dense(output_shape[-1]))

model.compile(loss='mse',
              optimizer="Adam")

model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Esta arquitectura comple lo solicitado, pero es bastante limitada y rebuscada. En el futuro y otros ejemplos veremos la arquitectura tipo encoder-decoder que es más flexible y "simétrica" que la utilizada en este caso.

In [ ]:
hist = model.fit(X, y, epochs=500, validation_split=0.2, batch_size=5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist.history['loss']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['loss'], label='train')
sns.lineplot(x=epoch_count,  y=hist.history['val_loss'], label='valid')
plt.show()

In [ ]:
# Ensayo
x_test = [20, 25, 30]
y_test = [x+15 for x in x_test]

test_input = np.array([x_test])
test_input = test_input.reshape((1, 3, 1))
y_hat = model.predict(test_input, verbose=0)[0]

print("y_test:", y_test)
print("y_hat:", y_hat[0], y_hat[1], y_hat[2])

model.evaluate(test_input, np.array([y_test]))